In [ ]:
# Suppress warnings
import warnings
warnings.simplefilter('ignore')

import pyrcel as pm
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
P0 = 77500. # Pressure, Pa
T0 = 274.   # Temperature, K
S0 = -0.02  # Supersaturation, 1-RH (98% here)

In [ ]:
mus = [0.025, 0.005, 0.035, 0.8]
sigmas = [2.8, 1.7, 2.2, 2.9]
Ns = [950, 850, 500, 600]

mus2 = [0.025, 0.005, 0.035, 0.8]
sigmas2 = [2.8, 1.7, 2.2, 2.9]
Ns2 = [95, 85, 50, 60]

mus3 = [0.025, 0.005, 0.035, 0.8]
sigmas3 = [2.8, 1.7, 2.2, 2.9]
Ns3 = [995, 1385, 2050, 3560]

In [ ]:
setup_multimode_aerosols = {"species": 
                            {"mus": {}, "sigmas": {}, "Ns": {}, "kappas": {}}
                            }

In [ ]:
aerosol =  pm.AerosolSpecies('sulfate', pm.MultiModeLognorm(mus=mus, sigmas=sigmas , Ns=Ns), kappa=0.45, bins=500)
aerosol2 =  pm.AerosolSpecies('sea_salt', pm.MultiModeLognorm(mus=mus2, sigmas=sigmas2 , Ns=Ns2), kappa=1.2, bins=500)
aerosol3 =  pm.AerosolSpecies('mos', pm.MultiModeLognorm(mus=mus3, sigmas=sigmas3 , Ns=Ns3), kappa=0.12, bins=500)

In [ ]:
aerosol

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
ax.grid(False, "minor")

ax.bar(aerosol3.rs[:-1], aerosol3.Nis*1e-6, np.diff(aerosol3.rs), color='green', label="a mixed mode of small particles")
ax.bar(aerosol.rs[:-1], aerosol.Nis*1e-6, np.diff(aerosol.rs), color='blue', label="sulfate")
ax.bar(aerosol2.rs[:-1], aerosol2.Nis*1e-6, np.diff(aerosol2.rs), color='red', label="sea salt")

ax.semilogx()
ax.set_xlabel("Aerosol dry radius, micron")
ax.set_ylabel("Aerosl number conc., cm$^{-3}$")
ax.legend(loc='upper right')

In [ ]:
initial_aerosols = [aerosol2, aerosol3]
V = 1.0 # updraft speed, m/s

dt = 1.0 # timestep, seconds
t_end = 250./V # end time, seconds... 250 meter simulation

model = pm.ParcelModel(initial_aerosols, V, T0, S0, P0, console=False, accom=0.3)
parcel_trace, aerosol_traces = model.run(t_end, dt, solver='cvode')

In [ ]:
fig, [axS, axA] = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

axS.plot(parcel_trace['S']*100., parcel_trace['z'], color='k', lw=2)
axT = axS.twiny()
axT.plot(parcel_trace['T'], parcel_trace['z'], color='r', lw=1.5)

#Smax = parcel_trace['S'].max()*100

#axS.set_xlim(0, 0.7)
#axS.set_ylim(0, 250)

#axT.set_xticks([270, 271, 272, 273, 274])
axT.xaxis.label.set_color('red')
axT.tick_params(axis='x', colors='red')

axS.set_xlabel("Supersaturation, %")
axT.set_xlabel("Temperature, K")
axS.set_ylabel("Height, m")

#sulf_array = aerosol_traces['sulfate'].values
sulf_array = aerosol_traces['mos'].values
sea_array = aerosol_traces['sea_salt'].values

ss = axA.plot(sulf_array[:, ::10]*1e6, parcel_trace['z'], color='b', label="sulfate")
sa = axA.plot(sea_array*1e6, parcel_trace['z'], color='r', label="sea salt")
#axA.semilogx()
#axA.set_xlim(1e-2, 10.)
#axA.set_xticks([1e-2, 1e-1, 1e0, 1e1], [0.01, 0.1, 1.0, 10.0])
axA.legend([ss[0], sa[0]], ['sulfate', 'sea salt'], loc='upper right')
axA.set_xlabel("Droplet radius, micron")

for ax in [axS, axA, axT]:
    ax.grid(False, 'both', 'both')

### Dirichlet process mixtures
https://pymc3-testing.readthedocs.io/en/rtd-docs/notebooks/dp_mix.html

In [ ]:
import scipy as sp
x_plot = np.linspace(-3, 3, 200)

N = 5
K = 30

alpha = 2
P0 = sp.stats.norm
f = lambda x, theta: sp.stats.norm.pdf(x, theta, 0.3)

beta = sp.stats.beta.rvs(1, alpha, size=(N, K))
w = np.empty_like(beta)
w[:, 0] = beta[:, 0]
w[:, 1:] = beta[:, 1:] * (1 - beta[:, :-1]).cumprod(axis=1)

theta = P0.rvs(size=(N, K))

dpm_pdf_components = f(x_plot[np.newaxis, np.newaxis, :], theta[..., np.newaxis])
dpm_pdfs = (w[..., np.newaxis] * dpm_pdf_components).sum(axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(x_plot, dpm_pdfs.T, c='gray');

ax.set_yticklabels([]);

fig, ax = plt.subplots(figsize=(8, 6))

ix = 1

ax.plot(x_plot, dpm_pdfs[ix], c='k', label='Density');
ax.plot(x_plot, (w[..., np.newaxis] * dpm_pdf_components)[ix, 0],
        '--', c='k', label='Mixture components (weighted)');
ax.plot(x_plot, (w[..., np.newaxis] * dpm_pdf_components)[ix].T,
        '--', c='k');

ax.set_yticklabels([]);
ax.legend(loc=1);